<a href="https://colab.research.google.com/github/xbadiam/Energy_consumption_steel_industry/blob/main/notebooks/Energy_consumption_steel_industry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Energy consumption of the steel industry

## 1. Context

This company produces several types of coils, steel plates, and iron plates. The information on electricity consumption is held in a cloud-based system. The information on energy consumption of the industry is stored on the website of the Korea Electric Power Corporation (pccs.kepco.go.kr), and the perspectives on daily, monthly, and annual data are calculated and shown.

### 1.1. Attribute Information:

* Date Continuous-time data taken on the first of the month
* Usage_kWh Industry Energy Consumption Continuous kWh
* Lagging Current reactive power Continuous kVarh
* Leading Current reactive power Continuous kVarh
* CO2 Continuous ppm
* NSM Number of Seconds from midnight Continuous S
* Week status Categorical (Weekend (0) or a Weekday(1))
* Day of week Categorical Sunday, Monday : Saturday
* Load Type Categorical Light Load, Medium Load, Maximum Load

### 1.2. Acknowledgements

This dataset is sourced from the UCI Machine Learning Repository

### 1.3. Inspiration
Which times of the year is the most energy consumed?
What patterns can we identify in energy usage?

## 2. Imports

In [ ]:
!rm -rf /content/Energy_consumption_steel_industry

! git clone https://github.com/xbadiam/Energy_consumption_steel_industry.git

In [ ]:
try:
  import skforecast
except:
  !pip install skforecast
  import skforecast

In [ ]:
# Data processing
# ==============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# graphic
# ==============================================================================
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Modelado y Forecasting
# ==============================================================================
from lightgbm import LGBMRegressor
import skforecast
from skforecast.plot import calculate_lag_autocorrelation, plot_residuals
from skforecast.recursive import ForecasterEquivalentDate, ForecasterRecursive
from skforecast.model_selection import TimeSeriesFold, bayesian_search_forecaster, backtesting_forecaster
from skforecast.preprocessing import RollingFeatures
from statsmodels.tsa.stattools import kpss, adfuller

## 2. Funtions


In [ ]:
# Function to run ADF Test and print results
def adf_test(series):
  ADF_result = adfuller(series)
  print(f'p-value: {ADF_result[1]}')
  if ADF_result[1] < 0.05:
     print("The time series is stationary (we Reject the null hypothesis H0).")
  else:
    print("The time series is non-stationary (we cannot reject the null hypothesis H0).")
  print('\n')


## 3. Load data

In [ ]:
data = pd.read_csv('/content/Energy_consumption_steel_industry/inputs/Steel_industry_data.csv')


In [ ]:
data.head(10)

In [ ]:
# Dataset information
data.info()


In [ ]:
# Statistical summary
print('Statistical Summary of Numerical Features:')
print('-' * 50)
data.describe().T

In [ ]:
# Preprocesado de datos (estableciendo índice y frecuencia)
# ==============================================================================

data['date'] = pd.to_datetime(data['date'], dayfirst=True, errors='coerce')
data = data.set_index('date')
data = data.asfreq('15min')
data = data.sort_index()

In [ ]:
start_date = data.index.min()
end_date = data.index.max()
date_range_complete = pd.date_range(start=start_date, end=end_date, freq=data.index.freq)

# Detectar si faltan fechas
missing_dates = date_range_complete.difference(data.index)

print(f"Complete index: {len(missing_dates) == 0}")
print(f"Missing dates: {len(missing_dates)}")
print(f"Rows with missing values: {data.isnull().any(axis=1).mean():.2%}")

In [ ]:
# Extract temporal features from date
data['year'] = data.index.year
data['month'] = data.index.month
data['day'] = data.index.day
data['hour'] = data.index.hour
data['minute'] = data.index.minute
data['day_of_year'] = data.index.dayofyear
data['week_of_year'] = data.index.isocalendar().week
data['quarter'] = data.index.quarter

## 4. Data division

the dataset starts on 2018-01-01 00:00:00 and ends on 2018-12-31 23:45:00. In addition, to optimize the model's hyperparameters and evaluate its predictive performance, the data is divided into three sets: training, validation and test.

In [ ]:
# 9. Dividir tren / test
split = int(len(data)*0.8)
data_train = data.iloc[:split]
data_test = data.iloc[split:]

In [ ]:
print(f"Fechas train      : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
#print(f"Fechas validacion : {datos_val.index.min()} --- {datos_val.index.max()}  (n={len(datos_val)})")
print(f"Fechas test       : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

## 5. Graphic exploration

### 5.1. Time series chart

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_train.index, y=data_train['Usage_kWh'], mode='lines', name='Train'))
#fig.add_trace(go.Scatter(x=data_val.index, y=datos_val['Demand'], mode='lines', name='Validation'))
fig.add_trace(go.Scatter(x=data_test.index, y=data_test['Usage_kWh'], mode='lines', name='Test'))
fig.update_layout(
    title  = 'Actual Usage (kWh)',
    xaxis_title="Date",
    yaxis_title="Usage (MWh)",
    legend_title="Particion:",
    width=800,
    height=400,
    margin=dict(l=20, r=20, t=35, b=20),
    legend=dict(orientation="h", yanchor="top", y=1, xanchor="left", x=0.001)
)
fig.show()

In [ ]:
# Gráfico serie temporal con zoom
# ==============================================================================
zoom = ('2018-04-01 14:00:00','2018-05-01 14:00:00')
fig = plt.figure(figsize=(8, 4))
grid = plt.GridSpec(nrows=8, ncols=1, hspace=0.6, wspace=0)
main_ax = fig.add_subplot(grid[1:3, :])
zoom_ax = fig.add_subplot(grid[5:, :])
data['Usage_kWh'].plot(ax=main_ax, c='black', alpha=0.5, linewidth=0.5)
min_y = min(data['Usage_kWh'])
max_y = max(data['Usage_kWh'])
main_ax.fill_between(zoom, min_y, max_y, facecolor='blue', alpha=0.5, zorder=0)
main_ax.set_xlabel('')
data.loc[zoom[0]: zoom[1]]['Usage_kWh'].plot(ax=zoom_ax, color='blue', linewidth=1)
main_ax.set_title(f'Usage (Kwh): {data.index.min()}, {data.index.max()}', fontsize=10)
zoom_ax.set_title(f'Demanda (Kwh): {zoom}', fontsize=10)
zoom_ax.set_xlabel('')
plt.subplots_adjust(hspace=1)

When zooming in on the time series, a clear weekly seasonality becomes evident, with higher consumption during weekdays (Monday to Friday) and lower or nothing consumption on weeknends. It is also observed that there is a strong correlation between the consumption of one day and that of the previous day.  

### 5.2. Stationarity chart

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(8, 5), sharex=False, sharey=True)
axs = axs.ravel()

# Distribution of usage per month
data['month'] = data.index.month
data.boxplot(column='Usage_kWh', by='month', ax=axs[0], flierprops={'markersize': 3, 'alpha': 0.3})
data.groupby('month')['Usage_kWh'].median().plot(style='o-', linewidth=0.8, ax=axs[0])
axs[0].set_ylabel('Usage_kWh')
axs[0].set_title('Usage per month', fontsize=9)

# Distribution of usage per day of week
data['week_day'] = data.index.day_of_week + 1
data.boxplot(column='Usage_kWh', by='week_day', ax=axs[1], flierprops={'markersize': 3, 'alpha': 0.3})
data.groupby('week_day')['Usage_kWh'].median().plot(style='o-', linewidth=0.8, ax=axs[1])
axs[1].set_ylabel('Usage (kWh)')
axs[1].set_title('Usage per day of the week', fontsize=9)

# Distribution of usage per hour of day
data['hour_day'] = data.index.hour + 1
data.boxplot(column='Usage_kWh', by='hour_day', ax=axs[2], flierprops={'markersize': 3, 'alpha': 0.3})
data.groupby('hour_day')['Usage_kWh'].median().plot(style='o-', linewidth=0.8, ax=axs[2])
axs[2].set_ylabel('Usage (kWh)')
axs[2].set_title('Distribution of usage per hour of day', fontsize=9)

# Distribusión de demanda por día de la semana y hora del día
mean_day_hour = data.groupby(["week_day", "hour_day"])["Usage_kWh"].mean()
mean_day_hour.plot(ax=axs[3])
axs[3].set(
    title       = "Usage average",
    xticks      = [i * 24 for i in range(7)],
    xticklabels = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"],
    xlabel      = "Day and hour",
    ylabel      = "Usage"
)

axs[3].title.set_size(10)

fig.suptitle("Stationarity charts", fontsize=12)
fig.tight_layout()

**Augmented Dickey-Fuller (ADF)**

The first step is to determine whether our random walk is stationary or not. We
know that since there are visible trends in our sequence, it is not stationary. Nevertheless, let’s apply the ADF test to make sure.

* If $p−valor <α$ ⇒ It is rejected ${H_0}$
* If $p−valor ≥ α$ ⇒ It is accepted ${H_0}$

### 5.3. Autocorrelation graphs

In [ ]:
# Gráfico autocorrelación
# ==============================================================================
fig, ax = plt.subplots(figsize=(5, 2))
plot_acf(data['Usage_kWh'], ax=ax, lags=72)
plt.show()

In [ ]:
# Partial autocorrelation graph
# ==============================================================================
fig, ax = plt.subplots(figsize=(5, 2))
plot_pacf(data['Usage_kWh'], ax=ax, lags=60)
plt.show()

In [ ]:
# Top 10 lags con mayor autocorrelación parcial absoluta
# ==============================================================================
calculate_lag_autocorrelation(
    data    = data['Usage_kWh'],
    n_lags  = 60,
    sort_by = "partial_autocorrelation_abs"
).head(10)

## 6. Modelo Baseline

In [ ]:
# Create a baseline: value of the same hour of the previous day
# ==============================================================================
forecaster = ForecasterEquivalentDate(
                 offset    = pd.DateOffset(days=1),
                 n_offsets = 1
             )

# Forescater's training
# ==============================================================================
end_train = str(data_train.index[-1])
forecaster.fit(y=data.loc[:end_train , 'Usage_kWh'])
forecaster

In [ ]:
# Backtesting
# ==============================================================================
end_test = str(data_test.index[-1])

cv = TimeSeriesFold(
        steps              = 24,
        initial_train_size = len(data.loc[:end_train]),
        refit              = False
)
metrica, predicciones = backtesting_forecaster(
                          forecaster = forecaster,
                          y          = data['Usage_kWh'],
                          cv         = cv,
                          metric     = 'mean_absolute_error'
                       )
metrica

## 7. Recursive autoregressive model

It's training a recursive autoregressive model

In [ ]:
# Create the forecaster
# ==============================================================================
window_features = RollingFeatures(stats=["mean"], window_sizes=24 * 3)

forecaster = ForecasterRecursive(
                 regressor       = LGBMRegressor(random_state=15926, verbose=-1),
                 lags            = 24,
                 window_features = window_features
             )

# Entrena el forecaster
# ==============================================================================
forecaster.fit(y=data.loc[:end_train, 'Usage_kWh'])
forecaster

### 7.1. Backtesting

In [ ]:
# Backtesting
# ==============================================================================
metrica, predicciones = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = data['Usage_kWh'],
                            cv         = cv,
                            metric     = 'mean_absolute_error',
                            verbose    = True,  # False para no mostrar info
                        )

In [ ]:
# Error backtest
# ==============================================================================
metrica

In [ ]:
# Predict
# ==============================================================================
predictions = forecaster.predict(steps=36)
predictions.head(3)

In [ ]:
# Gráfico prediccion vs valores reales
# ==============================================================================
fig = go.Figure()
trace1 = go.Scatter(x=data_test.index, y=data_test['Usage_kWh'], name="test", mode="lines")
trace2 = go.Scatter(x=predicciones.index, y=predicciones['pred'], name="prediction", mode="lines")
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    title="Predicción vs valores reales en test",
    xaxis_title="Date time",
    yaxis_title="Usage_kWh",
    width=800,
    height=400,
    margin=dict(l=20, r=20, t=35, b=20),
    legend=dict(orientation="h", yanchor="top", y=1.01, xanchor="left", x=0)
)
fig.show()